In [4]:
import redis

r = redis.Redis(host='redis', port=6379, db=0, decode_responses=True)

print("ping:", r.ping())

ping: True


In [5]:

player_key=f"player:{id}"

def add_player(id, name, country, score=0):
    player_data = {
        "name": name,
        "country": country,
        "score": score,
        "games_played": 0
    }

    r.hset(player_key, mapping=player_data)
    
    r.zadd("leaderboard", {id:score})

    print(f"[OK] Jugador añadido: {name} (ID: {id} con {score} puntos).")


In [6]:
add_player(1, "Victor", "España")
add_player(2, "Jose", "Francia")
add_player(3, "Carlos", "Portugal")

[OK] Jugador añadido: Victor (ID: 1 con 0 puntos).
[OK] Jugador añadido: Jose (ID: 2 con 0 puntos).
[OK] Jugador añadido: Carlos (ID: 3 con 0 puntos).


In [7]:
def update_score(id, points):
    
    if not r.exists(player_key):
        print(f"[Error] El jugador con ID {id}")
        return

    games_played = r.hget(player_key, "games_played")
    games_played_up = int(games_played)+1
    r.hset(player_key, "games_played", games_played_up)

    score= r.hget(player_key, "score")
    new_score= int(score) + points
    r.hset(player_key, "score", new_score)

    r.zadd("leaderboard", {id:new_score})

    print(f"[OK] Jugador {id} actualizo {points} puntos).")
    print(f"Nueva puntuacion{score}. partidos jugados")

In [8]:
update_score(1,50)
update_score(2,75)

[OK] Jugador 1 actualizo 50 puntos).
Nueva puntuacion0. partidos jugados
[OK] Jugador 2 actualizo 75 puntos).
Nueva puntuacion50. partidos jugados


In [9]:
def player_info(id):
    return r.hgetall(player_key)

    if not info:
        print(f"No encontrado")
        return None
    print()
    for key,value in info.items():
        print(f" > {key.capitalize()}: {value}")
    return info

In [10]:
def show_top_players(n):
    print(f"\n---TOP {n} JUGADORES")
    top = r.zrevrange("leaderboard", 0, n-1, withscores= True)

    if not top:
        print("Ranking vacio")
        return

    for i, (id,score) in enumerate(top):
        res=r.hgetall(player_key)

    print(f" #{i+1}: {res['name']}({res['country']})")
    print(f" ID :{id}| Puntuación:{int(res['score'])}")
    print("  "+ "-"*20)
    

In [11]:
show_top_players(2)


---TOP 2 JUGADORES
 #2: Carlos(Portugal)
 ID :1| Puntuación:125
  --------------------


In [20]:
from datetime import date

def get_date_key(date_obj):
    return f"unique:players:{date_obj.isoformat()}"

def register_login(player_id):
    today_key = get_date_key(date.today())

    changed = r.pfadd(today_key,player_id)

    if changed:
        print(f"[Login] {player_id} registrado por primera vez hoy")
    else:
        print(f"[Login] {player_id} ya habiainiciado sesion hoy")

In [21]:
register_login(2)

[Login] 2 registrado por primera vez hoy


In [23]:
def count_unique_logins(date_str):

    key = f"unique:players:{date_str}"
    
    count =r.pfcount(key)

    print(f"\n[Reporte Jugadores unicos(aprox.) el {date.str}: {count}")
    return count

In [24]:
def weekly_report(dates_list):
    keys_to_merge = {f"unique:players:{d}" for d in dates_list}

    report_key ="unique:players:week_report" # Clave temporal

    r.pfmerge(report_key, *keys_to_merge)

    total_unique = r.pfcount(report_key)

    print(f"\n Reporte Semanal (Dias: {len(dates_list)})")
    print(f"Total de jugadores unicos(aprox.) {total_unique}")

    r.delete(report_key) # Borrar clave temporal

    return total_unique

In [25]:
weekly_report(["2025-11-10","2025-11-11"])


 Reporte Semanal (Dias: 2)
Total de jugadores unicos(aprox.) 0


0